In [46]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime

import pandas as pd
import numpy as np

In [ ]:
import os
os.getcwd()

In [ ]:
%cd ../..

In [ ]:
from dotenv import load_dotenv

load_dotenv('bot/.env')

In [ ]:


# no keys required for crypto data
# client = CryptoHistoricalDataClient()

# request_params = CryptoBarsRequest(
#                         symbol_or_symbols=[
#                             "BTC/USD",
#                             "ETH/USD",
#                             "ADA/USD"
#                         ],
#                         timeframe=TimeFrame.Day,
#                         start=datetime(2022, 1, 1)
#                  )

# bars = client.get_crypto_bars(request_params)

In [ ]:
%run ./bot/db/main_db.py

In [ ]:

db = FinanceDB.Instance(schema='crypto').pg_db

In [ ]:
df = db.query('select * from information_schema.tables limit 1')
df

In [ ]:
# db.df_to_db(bars.df, name='crypto_bars')

In [ ]:
import sqlalchemy
sqlalchemy.__version__

In [ ]:
df = db.query_df("select * from finance.crypto.crypto_bars where symbol='ETH/USD' ORDER BY timestamp;")
df

In [13]:
df = (
    df
        .assign(**{
        'open-close': lambda x: x['open'] - x['close'],
        'high-low': lambda x: x['high'] - x['low'],
        })
        .dropna()
        .set_index('timestamp')
)
df

,symbol,open,high,low,close,volume,trade_count,vwap,open-close,high-low
timestamp,,,,,,,,,,
2022-01-01 06:00:00+00:00,ETH/USD,3737.79,3784.20,3674.37,3745.73,5567.407470,20524.0,3739.684582,-7.94,109.83
2022-01-02 06:00:00+00:00,ETH/USD,3744.14,3857.43,3731.18,3812.00,5407.467570,21175.0,3799.932562,-67.86,126.25
2022-01-03 06:00:00+00:00,ETH/USD,3813.53,3853.08,3681.79,3741.63,7290.358970,26467.0,3762.843739,71.90,171.29
2022-01-04 06:00:00+00:00,ETH/USD,3741.94,3893.47,3735.94,3804.28,8872.221780,30032.0,3822.980393,-62.34,157.53
2022-01-05 06:00:00+00:00,ETH/USD,3804.28,3848.00,3399.00,3458.12,18024.777020,52361.0,3606.037685,346.16,449.00
...,...,...,...,...,...,...,...,...,...,...
2023-05-21 05:00:00+00:00,ETH/USD,1817.63,1819.36,1791.64,1803.86,8294.515846,14746.0,1808.071841,13.77,27.72
2023-05-22 05:00:00+00:00,ETH/USD,1803.90,1869.00,1803.67,1856.44,12464.067194,23728.0,1825.339297,-52.54,65.33
2023-05-23 05:00:00+00:00,ETH/USD,1856.70,1862.04,1816.44,1829.49,12540.396888,22983.0,1848.831277,27.21,45.60


In [14]:
X = df[['open-close', 'high-low']]
X.head()

,open-close,high-low
timestamp,,
2022-01-01 06:00:00+00:00,-7.94,109.83
2022-01-02 06:00:00+00:00,-67.86,126.25
2022-01-03 06:00:00+00:00,71.90,171.29
2022-01-04 06:00:00+00:00,-62.34,157.53
2022-01-05 06:00:00+00:00,346.16,449.00


In [15]:
# The logic is that if the tomorrow’s closing price is greater than today’s closing price, then we will buy, else we will sell.
# We will store +1 for the buy signal and -1 for the sell signal. We will store the target variable in a variable ’Y’.
import numpy as np
Y = np.where(df['close'].shift(-1) > df['close'], 1, -1)
Y

array([ 1, -1,  1, -1, -1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1,  1,  1, -1,  1,  1,
       -1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1,
        1, -1,  1,  1, -1, -1,  1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1,
        1, -1, -1, -1,  1,  1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1,
        1, -1,  1, -1,  1, -1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1,  1,
       -1,  1, -1, -1,  1,  1, -1, -1, -1,  1, -1,  1, -1,  1,  1, -1, -1,
        1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1,  1,  1, -1,
       -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1, -1,
        1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,
       -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1,  1,  1,
       -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1, -1,
       -1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1,
       -1,  1, -1, -1, -1

In [16]:
# # Splitting the dataset
split_percentage = 0.7
split = int(split_percentage*len(df))
X_train = X[:split]
Y_train = Y[:split]
X_test = X[split:]
Y_test = Y[split:]

In [17]:
# instantiate k-nearest classifier. Here we are using ‘k =15’, you may vary the value of k and notice the change in result.
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)

# fit the model
knn.fit(X_train, Y_train)

# calculate accuracy
accuracy_train, accuracy_test = knn.score(X_train, Y_train), knn.score(X_test, Y_test)

print(f'Train Accuracy: {accuracy_train}')
print(f'Test Accuracy: {accuracy_test}')


Train Accuracy: 0.6162464985994398
Test Accuracy: 0.5751633986928104


In [18]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [19]:
# create a function to calculate the lorentzian distance of 2 vectors

def lorentzian_distance(x, y):
    return np.log(1 + np.sum(np.abs(x - y)))



In [20]:
# instantiate k-nearest classifier. Here we are using ‘k =15’, you may vary the value of k and notice the change in result.
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3, metric=lorentzian_distance)

# fit the model
knn.fit(X_train, Y_train)

# calculate accuracy
accuracy_train, accuracy_test = knn.score(X_train, Y_train), knn.score(X_test, Y_test)

print(f'Train Accuracy: {accuracy_train}')
print(f'Test Accuracy: {accuracy_test}')

Train Accuracy: 0.6358543417366946
Test Accuracy: 0.5228758169934641


In [21]:
from annoy import AnnoyIndex
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create an Annoy index for approximate nearest neighbor search
num_dimensions = len(X.columns)  # Number of dimensions for the feature vectors
num_trees = 10  # Number of trees in the Annoy index
annoy_index = AnnoyIndex(num_dimensions)

/tmp/ipykernel_14704/1787675473.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  annoy_index = AnnoyIndex(num_dimensions)


In [23]:
# Add data vectors to the Annoy index
for i, vector in enumerate(X.values):
    annoy_index.add_item(i, vector)
annoy_index.build(num_trees)

True

In [26]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=split_percentage, random_state=42)


In [60]:
# Find nearest neighbors' indices for training and testing data
k_neighbors = 1  # Number of nearest neighbors to consider
train_indices = [annoy_index.get_nns_by_vector(vector, k_neighbors) for vector in X_train.values]
test_indices = [annoy_index.get_nns_by_vector(vector, k_neighbors) for vector in X_test.values]


In [61]:
# Flatten the indices to 1D arrays
train_indices = [index for sublist in train_indices for index in sublist]
test_indices = [index for sublist in test_indices for index in sublist]


In [62]:
# Get the corresponding labels for training and testing indices
train_labels = Y[train_indices]
test_labels = Y[test_indices]

In [63]:
# Initialize and train the KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=k_neighbors)
model.fit(X_train, train_labels)

KNeighborsClassifier(n_neighbors=1)

In [65]:
# Make predictions on the test set
Y_pred = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(test_labels, Y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5210084033613446
